In [1]:
# Download the respective file
!gdown 1DUlkaMCJzSMxrf_gMP1aQaVn6Vmt-3mi

Downloading...
From: https://drive.google.com/uc?id=1DUlkaMCJzSMxrf_gMP1aQaVn6Vmt-3mi
To: /content/Lung_segmentation.zip
100% 367M/367M [00:04<00:00, 91.5MB/s]


In [2]:
# Unzip the contents of the file
!unzip Lung_segmentation.zip

Archive:  Lung_segmentation.zip
   creating: Test/
   creating: Test/Images/
 extracting: Test/Images/1000.png    
 extracting: Test/Images/801.png     
 extracting: Test/Images/802.png     
 extracting: Test/Images/803.png     
 extracting: Test/Images/804.png     
 extracting: Test/Images/805.png     
 extracting: Test/Images/806.png     
 extracting: Test/Images/807.png     
 extracting: Test/Images/808.png     
 extracting: Test/Images/809.png     
 extracting: Test/Images/810.png     
 extracting: Test/Images/811.png     
 extracting: Test/Images/812.png     
 extracting: Test/Images/813.png     
 extracting: Test/Images/814.png     
 extracting: Test/Images/815.png     
 extracting: Test/Images/816.png     
 extracting: Test/Images/817.png     
 extracting: Test/Images/818.png     
 extracting: Test/Images/819.png     
 extracting: Test/Images/820.png     
 extracting: Test/Images/821.png     
 extracting: Test/Images/822.png     
 extracting: Test/Images/823.png     
 extracting

In [3]:
# Import the required libraries
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image

In [4]:
# Import the required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

In [5]:
# Import the required libraries
from torch.utils.data import Dataset
import torch.utils.data as utils
from torchvision import transforms
import torch.nn.functional as F

In [6]:
# Import the required libraries
import torch
from tqdm import tqdm

In [7]:
# The segmentation dataset class
class SegmentationDataset(Dataset):
   # root directory and data transformation
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.images_dir = os.path.join(root_dir, 'Images')
        self.masks_dir = os.path.join(root_dir, 'Masks')
        self.image_filenames = os.listdir(self.images_dir)

    def __len__(self):
        return len(self.image_filenames)
#retrieves the corresponding image and mask
    def __getitem__(self, idx):
        img_name = os.path.join(self.images_dir, self.image_filenames[idx])
        mask_name = os.path.join(self.masks_dir, self.image_filenames[idx])
#convert the image to RGB format and the mask to grayscale ('L') format.
        image = Image.open(img_name).convert('RGB')
        mask = Image.open(mask_name).convert('L')
#apply the specified data transformation to the image
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image,mask


In [8]:
# transformation pipeline for images in a segmentation dataset
#Image transformations
transform = transforms.Compose([transforms.Resize(256),transforms.ToTensor()])

In [9]:
# Initialization of the dataset
dataset = SegmentationDataset(root_dir='Train/', transform=transform)

In [10]:
# the test dataset
test_dataset = SegmentationDataset('Test',transform=transform)

In [11]:
# the test dataloader
test_dataloader = torch.utils.data.DataLoader(test_dataset,batch_size=1,shuffle=False)

In [12]:
# load data in batches
batch_size=8

In [13]:
# loads data in batches, shuffles the data and makes it convenient for training neural networks
train_dataloader= torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [14]:
# sequential combination of networks
def conv_bn_relu(in_channels, out_channels, kernel, stride, padding):
    return nn.Sequential(
        # convolution operation
        nn.Conv2d(in_channels, out_channels, kernel, stride=stride, padding=padding),
        # Batch normalization
        nn.BatchNorm2d(out_channels),
        # non-linear activation function
        nn.ReLU(inplace=True),
    )

In [15]:
# Define the custom neural network architecture
# The architecture is designed for semantic segmentation and is built upon the ResNet50 backbone
# ResNet50 backbone for feature extraction and decoder layers for upsampling and refining features
class CustomSegment(nn.Module):
  # define number of output classes
    def __init__(self, out_class):
        super().__init__()

# Base ResNet50 model
        self.base_model = models.resnet50(pretrained=True)
        self.base_layers = list(self.base_model.children())
        self.reslayer0 = nn.Sequential(*self.base_layers[:4])
        self.reslayer1 = nn.Sequential(self.base_layers[4])
        self.reslayer2 = nn.Sequential(self.base_layers[5])
        self.reslayer3 = nn.Sequential(self.base_layers[6])
        self.reslayer4 = nn.Sequential(self.base_layers[7])

# Upsampling layers
        self.upsample2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.upsample4 = nn.Upsample(scale_factor=4, mode='bilinear', align_corners=True)
        self.up_convt = nn.Sequential(nn.ConvTranspose2d(1280,1024,kernel_size=4,stride=2,padding=1),
                                      nn.BatchNorm2d(1024),
                                      nn.ReLU(inplace=True))

# Decoder layers
        self.declayer1 = conv_bn_relu(256,256,3,1,1)
        self.declayer2 = conv_bn_relu(512,256,3,1,1)
        self.declayer3 = conv_bn_relu(1024,512,3,1,1)
        self.declayer4 = conv_bn_relu(2048,512,3,1,1)

# Last layers
        self.lastlayer1 = conv_bn_relu(1280,512,3,1,1)
        self.lastlayer2 = conv_bn_relu(512,256,1,1,0)
        self.lastlayer3 = conv_bn_relu(256,out_class,1,1,0)
        # Final convolution layer
        self.conv_last = nn.Conv2d(64, out_class, 1)

    def forward(self, x):
# Feature extraction using ResNet50
        x0 = self.reslayer0(x)
        x1 = self.reslayer1(x0)
        x2 = self.reslayer2(x1)
        x3 = self.reslayer3(x2)
        x4 = self.reslayer4(x3)
# Decoder layers
        dec1 = self.declayer1(x1)
        dec2 = self.declayer2(x2)
        dec3 = self.declayer3(x3)
        dec4 = self.declayer4(x4)

 # Upsampling layers
        up_dec3 = self.upsample2(dec3)
        up_dec4 = self.upsample4(dec4)

# Concatenate and upsample
        concat1 = torch.cat([dec2,up_dec3,up_dec4],dim=1)
        up_concat1 = self.up_convt(concat1)
        concat2 = torch.cat([dec1,up_concat1],dim=1)
# Last layers
        lastlayer1 = self.lastlayer1(concat2)
        lastlayer2 = self.lastlayer2(lastlayer1)
        lastlayer3 = self.lastlayer3(lastlayer2)
# Upsample the final output
        final_output =  self.upsample4(lastlayer3)
        return final_output
model = CustomSegment(out_class=2)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 106MB/s]


In [16]:
# set up the loss function
criterion = nn.CrossEntropyLoss()

In [17]:
# set up the SGD Optimizer
optimizer = torch.optim.SGD(model.parameters(), weight_decay=1e-4, lr = 0.001, momentum=0.9)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.001
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001
)


In [19]:
# Determine Device Availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
# move model to device
model = model.to(device)
print(model)

CustomSegment(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
 

In [23]:
# Performance of segmentation models by measuring the overlap between predicted and ground truth masks
# Calculate the IoU (Intersection over Union)
def iou_calculation(preds, masks):
    intersection = torch.logical_and(preds, masks).sum().item()
    union = torch.logical_or(preds, masks).sum().item()
    iou = intersection / union if union != 0 else 0.0
    return iou

In [ ]:
phase='train'
for epoch in range(25):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    running_iou = 0.0
# Iterate over batches in the training dataloader
    for inputs, masks in tqdm(train_dataloader, total=len(train_dataloader)):
      # Move the input data and masks to the specified device
        inputs = inputs.to(device)
        masks = masks.to(device).long()
        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == 'train'):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            masks = masks.squeeze(1)
# Calculate the loss using the specified criterion
            loss = criterion(outputs, masks)
            iou = iou_calculation(preds, masks)
            running_iou = running_iou + iou
# If in the training phase, performs backpropagation and updates the model parameters
            if phase == 'train':
                loss.backward()
                optimizer.step()
        running_loss = running_loss + loss.item() * inputs.size(0)
        running_corrects = running_corrects + torch.sum(preds == masks.data)
# epoch metrics
    epoch_loss = running_loss / len(train_dataloader.dataset)
    epoch_acc = running_corrects.double() / len(train_dataloader.dataset)
    epoch_iou = running_iou / len(train_dataloader)
# Print metrics for the epoch
    print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} IoU : {epoch_iou:.4f}')



100%|██████████| 100/100 [01:30<00:00,  1.10it/s]


train Loss: 0.4224 Acc: 59221.2900 IoU : 0.7578


100%|██████████| 100/100 [01:23<00:00,  1.20it/s]


train Loss: 0.3403 Acc: 62582.3925 IoU : 0.8572


100%|██████████| 100/100 [01:22<00:00,  1.21it/s]


...

100%|██████████| 100/100 [01:21<00:00,  1.23it/s]


train Loss: 0.1027 Acc: 64062.4775 IoU : 0.9249


100%|██████████| 100/100 [01:22<00:00,  1.22it/s]


train Loss: 0.0996 Acc: 64097.1588 IoU : 0.9267


 55%|█████▌    | 55/100 [00:45<00:38,  1.16it/s]

In [ ]:
# save the state dictionary of a PyTorch model
torch.save(model.state_dict(), 'state.pth')

In [ ]:
predict=[]
test_mask_list=[]

In [ ]:
with torch.no_grad():
  test_iou=0.0
  test_accuracy = 0.0
  test_loss=0.0
  # the model evaluation
  model.eval()

In [ ]:
  # Iterating through the test dataset
  for inputs,masks in tqdm(test_dataloader,total=len(test_dataloader)):

    inputs = inputs.to(device)
    masks = masks.to(device).long()
    outputs = model(inputs)

    _, preds = torch.max(outputs, 1)
    masks = masks.squeeze(1)

    loss = criterion(outputs, masks)
    iou = iou_calculation(preds, masks)
    # test iou
    test_iou = test_iou + iou
    # test loss
    test_loss = test_loss + loss.item()
    predict.append(outputs.cpu().numpy())
    test_mask_list.append(masks.cpu().numpy())

In [ ]:
# print iou
  print(test_iou/len(test_dataloader))

100%|██████████| 200/200 [00:12<00:00, 16.61it/s]

0.8707558079590889


In [ ]:
#Write a 2-page report using LaTex and upload your paper to ResearchGate or Arxiv, and put your paper link here.

https://www.researchgate.net/publication/376267042_Lung_Segmentation_using_Deep_Learning